In [3]:
# Import libraries
import ipywidgets as ipw
import src.widgets as widgets
import src.utils as utils
import json
import os
from jsonschema import validate, ValidationError
from IPython.display import display, clear_output

In [ ]:
# Import schema
SCHEMA_FILE = "schema/record.schema.json"
if os.path.exists(SCHEMA_FILE):
    SCHEMA = utils.read_json(SCHEMA_FILE)
else:
    print("Schema is not available!")

# Import database
DATABASE_FOLDER = "/home/jovyan/aiidalab-records-app/data"
if os.path.exists(DATABASE_FOLDER):
    DATABASE_FILE = f"{DATABASE_FOLDER}/records.json"
    if os.path.exists(DATABASE_FILE):
        DATABASE = utils.read_json(DATABASE_FILE)
    else:
        DATABASE = []

In [ ]:
# Initialise widgets
search_by_name_textbox = utils.Text(
    placeholder = "Search by name (keyword)"
)

search_by_structure_file_uploader = utils.FileUpload(
    accept = '.cdxml',
    multiple = False
)

search_button = utils.Button(
    tooltip = "Click me",
    description = "Search records"
)

search_tab_widgets = utils.HBox(
    [
        search_by_name_textbox,
        search_by_structure_file_uploader,
        search_button
    ]
)

search_result_accordion = utils.Accordion()

search_records_widgets = utils.VBox(
    [
        search_tab_widgets,
        search_result_accordion
    ]
)

record_widgets = widgets.RecordObjectWidget()

save_records_button = utils.Button(
    tooltip = "Click me",
    description = "Save record"
)
record_tab_widgets = utils.VBox(
    [
        record_widgets,
        save_records_button
    ]
)

interface_tabs = utils.Tab()
interface_contents = [search_records_widgets, record_tab_widgets]
interface_titles = ["Search records", "Insert records"]
interface_tabs.children = interface_contents
for i, title in enumerate(interface_titles):
    interface_tabs.set_title(i, title)

In [ ]:
# Action functions
def save_data(b):
    record_data = record_widgets.get_widgets_data()
    try:
        validate(instance = record_data, schema = SCHEMA)
        DATABASE.append(record_data)
        utils.write_json(DATABASE, DATABASE_FILE)
        print("Record saved successfully.")
        
    except ValidationError as e:
        print("JSON is invalid!")
        print("Reason:", e.message)

def search_records(b):
    # Clear previous titles
    previous_num_records = len(search_result_accordion.children)
    for i in range(previous_num_records):
        search_result_accordion.set_title(i, "")
            
    name = search_by_name_textbox.value
    if name:
        records_list = []
        for record in DATABASE:
            for record_name in record["names"]:
                if record_name == name:
                    record_widgets = widgets.RecordObjectWidget()
                    record_widgets.load_widgets(record)
                    records_list.append(record_widgets)
        
        # Update current search results titles
        num_records = len(records_list)
        search_result_accordion.children = records_list
        for i in range(num_records):
            search_result_accordion.set_title(i, f"Record {i+1}")

In [ ]:
# Add actions to widgets
search_button.on_click(search_records)
save_records_button.on_click(save_data)

# Carbon Database

In [ ]:
interface_tabs